#Librerias y dependecias

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from ytmusicapi import YTMusic
import re
from tqdm import tqdm
import time
import random
import requests

In [2]:
# ============================================================
# AUTENTICACIÓN
# ============================================================

CLIENT_ID = "a86bf1cc39144f3ab03f0e42453a65a4"
CLIENT_SECRET = "01e435a598134bb5ae1efb343229eed5"

sp = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET
    )
)

yt = YTMusic()  # no requiere auth si no accedés a tu cuenta personal

# CREACION EL DATASET


## Definir tus listas

In [3]:
# --- SPOTIFY ---
playlistsspotify = [
    ("rock", "https://open.spotify.com/playlist/2XGHHAtKIkF2vra9n9ohCo"),
    ("blues", "https://open.spotify.com/playlist/4tuvViSICMHa50NAwGaOiz"),
    ("country", "https://open.spotify.com/playlist/66W297BG3yvKGkUuawESJo"),
    ("hyperpop", "https://open.spotify.com/playlist/2spbReDu7dcRW8sMXwMfqz"),
    ("kpop", "https://open.spotify.com/playlist/4r9P2LhsIwLHtWzpfk3H3M"),
    ("kpop", "https://open.spotify.com/playlist/58N7rQpPjyVA6KCBVlOJOd"),
    ("electronica", "https://open.spotify.com/playlist/4o6pV8avI1bS6sB4zHW3no"),
    ("pop", "https://open.spotify.com/playlist/5Sx1Y3RRSorf9FENwcg0Js"),
    ("regueton", "https://open.spotify.com/playlist/03sDEv7FN58Mb9CJOs1Tgn"),
    ("rock", "https://open.spotify.com/playlist/5ffh8oWUXPpuqQFu5AvU6a"),
    ("hiphop", "https://open.spotify.com/playlist/30A7kR1hbfnmmSFDXh3JBT"),
    ("jazz", "https://open.spotify.com/playlist/2hiw0p7Z6KJECOhrB4qkHZ"),
    ("clasica", "https://open.spotify.com/playlist/2Irt1QeunuKClnMypBvtbs")
]


# --- YOUTUBE MUSIC ---
playlistsyoutube = [
    ("clasica", "https://music.youtube.com/playlist?list=PLb14QDz_8YuSzy0wcFDz9TyIduyuAPFOx"),
    ("jazz", "https://music.youtube.com/playlist?list=PLb14QDz_8YuR20uMKjryzBzIKvxZEzftQ"),
    ("hiphop", "https://music.youtube.com/playlist?list=PLb14QDz_8YuSzN5-_XYmIk-Z1nH__Hmf0"),
    ("rock", "https://music.youtube.com/playlist?list=PLb14QDz_8YuQCGXnBubIDe336aCUiVf3c"),
    ("rock", "https://music.youtube.com/playlist?list=PLb14QDz_8YuRtwDNk4DTf9lLl3yV5agtY"),
    ("blues", "https://music.youtube.com/playlist?list=PLb14QDz_8YuS3g51nM-OTq5EfSDQriwrz"),
    ("country", "https://music.youtube.com/playlist?list=PLb14QDz_8YuQmU0F_sPB5m-7c0nO7bRWq"),
]





## Funciones para extraer datos

In [4]:
# --- EXTRACCIÓN ID YOUTUBE MUSIC ---
def obtener_id_youtube(url):
    match = re.search(r"list=([a-zA-Z0-9_-]+)", url)
    return match.group(1) if match else None

def extraer_youtube_playlist(url, genero):
    playlist_id = obtener_id_youtube(url)
    if not playlist_id:
        print(f"No se pudo obtener ID de {url}")
        return []

    try:
        playlist = yt.get_playlist(playlist_id)
    except Exception as e:
        print(f"Error en {url}: {e}")
        return []

    data = []
    for item in playlist.get("tracks", []):
        if "artists" in item:
            artista = item["artists"][0]["name"]
            tema = item["title"]
            data.append((artista, tema, genero, "YouTube Music"))
    return data

def extraer_spotify_playlist(playlist_url, genero):
    """
    Extrae todas las canciones de una playlist de Spotify usando paginación
    Incluye el spotify_id directamente
    """
    try:
        # Extraer el ID de la playlist de la URL
        playlist_id = playlist_url.split('playlist/')[1].split('?')[0]

        data = []
        offset = 0
        limit = 100  # Máximo permitido por la API

        while True:
            # Obtener un lote de canciones
            results = sp.playlist_tracks(playlist_id, limit=limit, offset=offset)

            if not results['items']:
                break  # No hay más canciones

            # Procesar las canciones del lote actual
            for item in results['items']:
                if item['track'] and item['track']['artists']:
                    artista = item['track']['artists'][0]['name']
                    tema = item['track']['name']
                    popularidad = item['track']['popularity']
                    spotify_id = item['track']['id']
                    fecha_lanzamiento = item['track']['album']['release_date']
                    año = int(fecha_lanzamiento.split('-')[0])

                    # Filtrar solo 2024 y 2025
                    if año in [2024, 2025]:
                        data.append([artista, tema, genero, popularidad, "Spotify", spotify_id, año])


            # Si no hay más páginas, salir del bucle
            if results['next'] is None:
                break

            offset += limit  # Avanzar al siguiente lote

        return data

    except Exception as e:
        print(f"Error al extraer playlist {playlist_url}: {e}")
        return []

## Construir el dataset

### Spotify

In [5]:
data_spotify = []
for genero, url in tqdm(playlistsspotify):
    data_spotify.extend(extraer_spotify_playlist(url, genero))

df_spotify = pd.DataFrame(data_spotify, columns=["artista", "tema", "genero", "popularidad","fuente", "spotify_id",'año'])
print(f"Spotify: {len(df_spotify)} canciones")
print(f"Con spotify_id: {df_spotify['spotify_id'].notna().sum()}")
df_spotify.head()

100%|██████████| 13/13 [00:22<00:00,  1.71s/it]

Spotify: 2207 canciones
Con spotify_id: 2207


,artista,tema,genero,popularidad,fuente,spotify_id,año
0,Kings of Leon,To Space,rock,59,Spotify,356x2OD1llA9NfOMXVmGSk,2025
1,Teen Jesus and the Jean Teasers,WONDERFUL,rock,42,Spotify,2KlnNibs8vbh1ulZ7124vI,2025
2,Mother Mother,HOT TO GO! - Spotify Singles,rock,46,Spotify,5HoBujicJSH3hEIeDfmpTB,2025
3,Hayley Williams,Showbiz,rock,55,Spotify,0HA0F6W8khJgVgZwSEdTgN,2025
4,Jimmy Eat World,Failure,rock,52,Spotify,3pnfUFrTc8i1hpsORDbx1T,2025


### Youtube

In [6]:
data_youtube = []
for genero, url in tqdm(playlistsyoutube):
    data_youtube.extend(extraer_youtube_playlist(url, genero))

df_youtube = pd.DataFrame(data_youtube, columns=["artista", "tema", "genero", "fuente"])
print(f"YouTube Music: {len(df_youtube)} canciones")
df_youtube.head()


100%|██████████| 7/7 [00:03<00:00,  2.31it/s]

YouTube Music: 438 canciones


,artista,tema,genero,fuente
0,James Quinn,Journey of Life,clasica,YouTube Music
1,Yannick Lowack,Silence,clasica,YouTube Music
2,Andrea Vanzo,Rebirth,clasica,YouTube Music
3,Hans Zimmer,"Interstellar Suite: Part 2, No Time for Caution",clasica,YouTube Music
4,Carlotta Dalia,Estudio sin luz,clasica,YouTube Music


### Consolidacion dataset Youtube/Spotify

In [7]:
df_total = pd.concat([df_spotify, df_youtube], ignore_index=True)

#eliminar duplicados por artista + tema
df_total.drop_duplicates(subset=["artista", "tema"], inplace=True)

print(f"Dataset total: {len(df_total)} canciones")

Dataset total: 2564 canciones


## Guardar el dataset final

In [11]:
df_total.to_csv("../data/raw/01_dataset_youtube_spotify.csv", index=False)


## Rellenando el dataset

### Función para buscar el ID

In [12]:
def obtener_spotify_id(artista, tema):
    """Busca el ID de Spotify combinando artista y tema."""
    query = f"track:{tema} artist:{artista}"
    try:
        result = sp.search(q=query, type="track", limit=1, market="AR")
        tracks = result.get("tracks", {}).get("items", [])
        if tracks:
            return tracks[0]["id"]
    except Exception:
        pass
    return None


### Aplicar sobre tu dataset

In [13]:
# Copiamos el DataFrame base y reiniciamos el índice
df = df_total.copy().reset_index(drop=True)

# Si la columna spotify_id no existe, crearla
if "spotify_id" not in df.columns:
    df["spotify_id"] = None

# Identificar las filas que necesitan búsqueda (sin ID)
filas_sin_id = df[df["spotify_id"].isna()].index

print(f"🔍 Canciones sin spotify_id: {len(filas_sin_id)} / {len(df)}")

# Buscar los IDs solo para las canciones que no lo tienen
for i in tqdm(filas_sin_id):
    artista = df.loc[i, "artista"]
    tema = df.loc[i, "tema"]

    spotify_id = obtener_spotify_id(artista, tema)
    df.loc[i, "spotify_id"] = spotify_id

    # Espera aleatoria para evitar rate limit (opcional, descomentar si es necesario)
    # time.sleep(0.05 + random.random() * 0.05)

# Guardar resultado completo
ruta_salida = "../data/interim/02_dataset_Y_S_con_spotify_ids.csv"
df.to_csv(ruta_salida, index=False, encoding="utf-8-sig")

print(f" Archivo guardado correctamente en: {ruta_salida}")
print(f" IDs totales encontrados: {df['spotify_id'].notna().sum()} / {len(df)}")
print(f"  - Desde playlists: {len(df) - len(filas_sin_id)}")
print(f"  - Mediante búsqueda: {df.loc[filas_sin_id, 'spotify_id'].notna().sum()}")


🔍 Canciones sin spotify_id: 407 / 2564


100%|██████████| 407/407 [02:05<00:00,  3.24it/s]

 Archivo guardado correctamente en: ../data/interim/02_dataset_Y_S_con_spotify_ids.csv
 IDs totales encontrados: 2454 / 2564
  - Desde playlists: 2157
  - Mediante búsqueda: 297


In [14]:
df.head()

,artista,tema,genero,popularidad,fuente,spotify_id,año
0,Kings of Leon,To Space,rock,59.0,Spotify,356x2OD1llA9NfOMXVmGSk,2025.0
1,Teen Jesus and the Jean Teasers,WONDERFUL,rock,42.0,Spotify,2KlnNibs8vbh1ulZ7124vI,2025.0
2,Mother Mother,HOT TO GO! - Spotify Singles,rock,46.0,Spotify,5HoBujicJSH3hEIeDfmpTB,2025.0
3,Hayley Williams,Showbiz,rock,55.0,Spotify,0HA0F6W8khJgVgZwSEdTgN,2025.0
4,Jimmy Eat World,Failure,rock,52.0,Spotify,3pnfUFrTc8i1hpsORDbx1T,2025.0


# Audiofeatures con Reccobeats

In [15]:
output_path = "../data/interim/03_dataset_con_audiofeatures_reccobeats.csv"

# Detectar columna que contiene los IDs
for c in df.columns:
    if "spotify" in c.lower() and "id" in c.lower():
        col_id = c
        break

if not col_id:
    raise ValueError("No se encontró una columna que contenga 'spotify' y 'id' en su nombre.")

spotify_ids = df[col_id].dropna().astype(str).unique().tolist()
print(f"✅ Dataset cargado: {len(df)} filas")
print(f"🎵 Se encontraron {len(spotify_ids)} Spotify IDs válidos")

# === FUNCIÓN PARA OBTENER FEATURES ===
def get_audio_features_batch(ids):
    ids_str = ",".join(ids)
    url = f"https://api.reccobeats.com/v1/audio-features?ids={ids_str}"
    headers = {"Accept": "application/json"}
    res = requests.get(url, headers=headers)
    if res.status_code != 200:
        print(f"⚠️ Error {res.status_code}: {res.text[:150]}")
        return []
    data = res.json().get("content", [])
    return data

# === DESCARGA EN BLOQUES ===
features = []
batch_size = 40  # ✅ la API permite entre 1 y 40

for i in range(0, len(spotify_ids), batch_size):
    batch = spotify_ids[i:i+batch_size]
    data = get_audio_features_batch(batch)
    if data:
        features.extend(data)
    print(f"Procesado bloque {i//batch_size + 1}/{len(spotify_ids)//batch_size + 1} ({len(data)} features)")
    time.sleep(0.3)  # evitar rate limit

# === CONVERTIR A DATAFRAME Y UNIR ===
if not features:
    print("⚠️ No se obtuvieron features de ningún track.")
else:
    df_features = pd.DataFrame(features)
    df_features["spotify_id"] = df_features["href"].str.extract(r"track/([a-zA-Z0-9]+)")
    df_final = df.merge(df_features, left_on=col_id, right_on="spotify_id", how="left")

    df_final.to_csv(output_path, index=False)
    print(f"✅ Archivo guardado en: {output_path}")
    print(f"🎧 Features obtenidas: {df_features.shape[0]}")


✅ Dataset cargado: 2564 filas
🎵 Se encontraron 2451 Spotify IDs válidos
Procesado bloque 1/62 (0 features)
Procesado bloque 2/62 (1 features)
Procesado bloque 3/62 (7 features)
Procesado bloque 4/62 (27 features)
Procesado bloque 5/62 (23 features)
Procesado bloque 6/62 (4 features)
Procesado bloque 7/62 (0 features)
Procesado bloque 8/62 (1 features)
Procesado bloque 9/62 (20 features)
Procesado bloque 10/62 (23 features)
Procesado bloque 11/62 (21 features)
Procesado bloque 12/62 (22 features)
Procesado bloque 13/62 (32 features)
Procesado bloque 14/62 (36 features)
Procesado bloque 15/62 (26 features)
Procesado bloque 16/62 (27 features)
Procesado bloque 17/62 (27 features)
Procesado bloque 18/62 (25 features)
Procesado bloque 19/62 (16 features)
Procesado bloque 20/62 (5 features)
Procesado bloque 21/62 (4 features)
Procesado bloque 22/62 (14 features)
Procesado bloque 23/62 (25 features)
Procesado bloque 24/62 (31 features)
Procesado bloque 25/62 (20 features)
Procesado bloque 26/

In [16]:
df_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2564 entries, 0 to 2563
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artista           2564 non-null   object 
 1   tema              2564 non-null   object 
 2   genero            2564 non-null   object 
 3   popularidad       2157 non-null   float64
 4   fuente            2564 non-null   object 
 5   spotify_id        2454 non-null   object 
 6   año               2157 non-null   float64
 7   id                865 non-null    object 
 8   href              865 non-null    object 
 9   acousticness      864 non-null    float64
 10  danceability      864 non-null    float64
 11  energy            864 non-null    float64
 12  instrumentalness  864 non-null    float64
 13  key               864 non-null    float64
 14  liveness          864 non-null    float64
 15  loudness          864 non-null    float64
 16  mode              864 non-null    float64


In [17]:
print(df_final["genero"].value_counts())

genero
rock           958
hiphop         277
jazz           263
hyperpop       199
kpop           194
pop            185
regueton       148
clasica         98
electronica     96
blues           76
country         70
Name: count, dtype: int64


# Filtrado de resultados TOP 100

In [ ]:
import pandas as pd

# 1️⃣ Eliminar registros con nulos en acousticness
df_filtrado = df_final.dropna(subset=["acousticness"]).copy()

# 🔹 Agregado: rellenar popularidad faltante con promedio por género
df_filtrado["popularidad"] = df_filtrado.groupby("genero")["popularidad"].transform(
    lambda x: x.fillna(x.mean())
)

# 2️⃣ Asegurar que popularidad sea numérica
df_filtrado["popularidad"] = pd.to_numeric(df_filtrado["popularidad"], errors="coerce")

# 3️⃣ Función auxiliar: top N por género
def top_por_genero(df, genero, n=100):
    subset = df[df["genero"].str.lower() == genero.lower()]
    subset_top = subset.sort_values("popularidad", ascending=False).head(n)
    return subset_top

# 4️⃣ Definir los géneros a limitar
generos_limitados = ["rock", "kpop", "regueton","hiphop","pop"]

# 5️⃣ Filtrar el resto (que no se tocan)
otros_generos = df_filtrado[~df_filtrado["genero"].str.lower().isin(generos_limitados)]

# 6️⃣ Tomar top 100 de los géneros seleccionados
top_rock = top_por_genero(df_filtrado, "rock", 150)
top_kpop = top_por_genero(df_filtrado, "kpop", 70)
top_regueton = top_por_genero(df_filtrado, "regueton", 100)
top_hiphop = top_por_genero(df_filtrado, "hiphop", 150)
top_pop = top_por_genero(df_filtrado, "pop", 50)

# 7️⃣ Unir todo: los tops + los géneros restantes sin cambios
df_final_filtrado = pd.concat([otros_generos, top_rock, top_kpop, top_regueton,top_hiphop,top_pop], ignore_index=True)

# 8️⃣ Resumen
print(df_final_filtrado["genero"].value_counts())
print(f"✅ Dataset final: {df_final_filtrado.shape[0]} filas")


# (Opcional) Guardar
df_final_filtrado.to_csv("../data/processed/04_dataset_filtrado.csv", index=False)


genero
rock           150
hiphop         150
regueton        90
kpop            70
electronica     61
country         56
pop             50
hyperpop        43
jazz            39
blues           18
clasica         16
Name: count, dtype: int64
✅ Dataset final: 743 filas
